# **Transfer Learning On Plant Seedlings Dataset**



---

This notebook is laid out as follows:



1.   **Part 1:** Loading,Transforming and One Hot Encoding 
2.   **Part 2:** Data Augmentation
3.  **Part 3:** Using Pre Trained models(Includes Saving and Loading)




## **Part 1: Loading,Transforming and One Hot Encoding**


In [ ]:
import os
import fnmatch
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.applications.vgg16 import preprocess_input
from keras.applications.inception_v3 import preprocess_input
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
np.random.seed(21)



import tensorflow as tf
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Activation,Dropout,GlobalAveragePooling2D
from keras.models import Sequential,Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam


a = '/kaggle/input/plant-seedlings-classification'

print(os.listdir(os.path.join(a,'train')))

In [ ]:
    from sklearn.metrics import accuracy_score


In [ ]:
def get_train_data(root):
    
    """Performs required pre processing on the input images and fetches data

    Args:
      root: Directory in which we are working

    Returns: 
      train_img: A numpy array consisting of train images
      train_y : A OHE numpy array of train labels
    """
    train_dir = (os.path.join(root,'train'))
    train_label = []
    train_img = []
    label2num = {'Loose Silky-bent':0, 'Charlock':1, 'Sugar beet':2, 'Small-flowered Cranesbill':3,
                 'Common Chickweed':4, 'Common wheat':5, 'Maize':6, 'Cleavers':7, 'Scentless Mayweed':8,
                 'Fat Hen':9, 'Black-grass':10, 'Shepherds Purse':11}
    for i in os.listdir(train_dir):
        label_number = label2num[i]
        new_path = os.path.join(train_dir,i)
        for j in fnmatch.filter(os.listdir(new_path), '*.png'):
            temp_img = image.load_img(os.path.join(new_path,j), target_size=(128,128))
            train_label.append(label_number)
            temp_img = image.img_to_array(temp_img)
            train_img.append(temp_img)
        print(i)
    train_img = np.array(train_img)

    train_y=pd.get_dummies(train_label)
    train_y = np.array(train_y)
    train_img=preprocess_input(train_img)
    
    return train_img,train_y



In [ ]:
train_img,train_y = get_train_data(a)
print('Training data shape: ', train_img.shape)
print('Training labels shape: ', train_y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.1, random_state=42)

## **Part 2: Data Augmentation**


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

datagen.fit(X_train)


## **Part 3- Transfer Learing**

### **1) VGG-16**

#### **Customizing VGG-16 for our problem statement**

In [ ]:
def vgg16_model(num_classes=None):
    
    """ Adding custom model to the VGG-16

    Args:
      num_classes: Number of layers in the final layer(Number of classes)

    Returns:
      model: Returns the custom model added to VGG
    """

    model = VGG16(weights='imagenet', include_top=False,input_shape=(128,128,3))
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    #model.layers[-2].outbound_node= []
    x=Conv2D(256, kernel_size=(2,2),strides=2)(model.output)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)    
    x=Conv2D(128, kernel_size=(2,2),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)

    model=Model(model.input,x)

    for layer in model.layers[:15]:

        layer.trainable = False


    return model

In [ ]:
from keras.applications.vgg16 import VGG16
from keras import backend as K
num_classes=12
model = vgg16_model(num_classes)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#### **Training VGG-16**

In [ ]:
from keras.callbacks import ModelCheckpoint
epochs = 10
batch_size = 32
model_checkpoint = ModelCheckpoint('vgg_weights.h5', monitor='val_accuracy', save_best_only=True,mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=0.000001)
#early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=7, verbose=0, mode='min', restore_best_weights=True)

model.fit(datagen.flow(X_train,Y_train),
          batch_size=128,
          epochs=20,
          verbose=1, shuffle=True, validation_data=(X_valid,Y_valid), callbacks=[model_checkpoint,reduce_lr])

In [ ]:
plt.style.use('seaborn')

# Accuracy History
def accuracy_curves(m):
    """ Plots the Train and validation Accuracy Curves

    Args:
      m: Model for which the curves are plotted

    Returns:
      
    """
    
    plt.figure(1, figsize=(16, 10))
    plt.plot(m.history.history['accuracy'])
    plt.plot(m.history.history['val_accuracy'])
    plt.title('Train and Validation Accuracy', fontsize = 16)
    plt.ylabel('Accuracy', fontsize = 14)
    plt.xlabel('Epoch', fontsize = 14)
    plt.legend(['Train', 'Test'], fontsize = 14)
    plt.show()
    
# Loss History

def loss_curves(m):
    """ Plots the Train and validation Loss Curves

    Args:
      m: Model for which the curves are plotted

    Returns:
      
    """
    plt.figure(2, figsize=(16, 10))
    plt.plot(m.history.history['loss'])
    plt.plot(m.history.history['val_loss'])
    plt.title('Train and Validation Loss', fontsize = 16)
    plt.ylabel('Loss', fontsize = 14)
    plt.xlabel('Epoch', fontsize = 14)
    plt.legend(['Train', 'Test'], fontsize = 14)
    plt.show()

    
from sklearn.metrics import classification_report

def class_report(m):
    """ Creates  Classification Report 

    Args:
      m: Model for which the report is generated

    Returns:
      
    """
    target_names = os.listdir(os.path.join(a,'train'))
    print(classification_report(Y_valid.argmax(axis=1), m.predict(X_valid).argmax(axis=1), target_names=target_names))
    
    
from sklearn.metrics import confusion_matrix

def conf_matrix(m):
    """ Creates  Confusion Matrix 

    Args:
      m: Model for which the confusion matrix is generated

    Returns:
      
    """
    target_names = os.listdir(os.path.join(a,'train'))
    cf_matrix = confusion_matrix(Y_valid.argmax(axis=1), np.round(m.predict(X_valid).argmax(axis=1),0))
    plt.figure(figsize=(20,20))
    sns.heatmap(cf_matrix/100, annot=True, xticklabels=target_names, yticklabels=target_names, cmap='Blues')

#### **Visualizing Loss and Accuracy(VGG-16)**

In [ ]:
accuracy_curves(model)

In [ ]:
loss_curves(model)

#### **Classification Report**

In [ ]:
class_report(model)

#### **Confusion matrix**

In [ ]:
conf_matrix(model)

#### Model Optimization Using TF-Lite(Post Training Dynamic range quantization) 

In [ ]:
import pathlib

In [ ]:
tflite_models_dir = pathlib.Path(os.path.join(os.getcwd(),'tflite_models'))
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
converter_vgg = tf.lite.TFLiteConverter.from_keras_model(model)


In [ ]:
# Convert to TF Lite without quantization
vgg16_tflite_file = tflite_models_dir/"vgg16.tflite"
vgg16_tflite_file.write_bytes(converter_vgg.convert())

In [ ]:
interpreter = tf.lite.Interpreter(model_path='./tflite_models/vgg16.tflite')
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
def run_tflite_model(tflite_file, test_image_indices):
    global X_valid

  # Initialize the interpreter
    interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    predictions = np.zeros((len(test_image_indices),), dtype=int)
    for i, test_image_index in enumerate(test_image_indices):
        test_image = X_valid[test_image_index]
        test_label = Y_valid[test_image_index]

        test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
        interpreter.set_tensor(input_details["index"], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details["index"])[0]

        predictions[i] = output.argmax()

    return predictions

In [ ]:
def evaluate_model(tflite_file, model_type):
    global X_valid
    global Y_valid

    test_image_indices = range(X_valid.shape[0])
    predictions = run_tflite_model(tflite_file, test_image_indices)
    
    predictions=pd.get_dummies(predictions)
    predictions = np.array(predictions)
    #print(predictions)
    
    accuracy = accuracy_score(y_true=Y_valid, y_pred=predictions) 

    #accuracy = (np.sum(Y_valid== predictions) * 100) / len(X_valid)

    return accuracy

In [ ]:
op = evaluate_model(vgg16_tflite_file, model_type="Float")
print("Accuracy of TFLite - VGG : {}".format(op))

#### Optimization of VGG Model with TensorRT

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt


In [ ]:
model.save('vgg_saved_model')

In [ ]:
print('Converting to TF-TRT FP32...')
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.FP32,
                                                               max_workspace_size_bytes=8000000000)

converter = trt.TrtGraphConverterV2(input_saved_model_dir='vgg_saved_model',
                                    conversion_params=conversion_params)
converter.convert()
converter.save(output_saved_model_dir='vgg_saved_model_TFTRT_FP32')
print('Done Converting to TF-TRT FP32')

In [ ]:
!saved_model_cli show --all --dir vgg_saved_model_TFTRT_FP32

In [ ]:
batch_size = 8
batched_input = np.zeros((batch_size, 128, 128, 3), dtype=np.float32)

for i in range(batch_size):
    img_path = '../input/plant-seedlings-classification/train/Common Chickweed'
    for j in os.listdir(img_path):
        img = image.load_img(os.path.join(img_path,j), target_size=(128, 128))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        batched_input[i, :] = x
batched_input = tf.constant(batched_input)
print('batched_input shape: ', batched_input.shape)

In [ ]:
import time

In [ ]:
def benchmark_tftrt(input_saved_model):
    saved_model_loaded = tf.saved_model.load(input_saved_model, tags=[tag_constants.SERVING])
    infer = saved_model_loaded.signatures['serving_default']

    N_warmup_run = 50
    N_run = 500
    elapsed_time = []

    for i in range(N_warmup_run):
        labeling = infer(batched_input)

    for i in range(N_run):
        start_time = time.time()
        labeling = infer(batched_input)
        #prob = labeling['probs'].numpy()
        end_time = time.time()
        elapsed_time = np.append(elapsed_time, end_time - start_time)
        if i % 50 == 0:
            print('Step {}: {:4.1f}ms'.format(i, (elapsed_time[-50:].mean()) * 1000))

    print('Throughput: {:.0f} images/s'.format(N_run * batch_size / elapsed_time.sum()))

### **2) ResNet50**

#### **Customizing ResNet50 for our problem statement**

In [ ]:
def resnet_model(num_classes=None):
    
    """ Adding custom model to the ResNet50

    Args:
      num_classes: Number of layers in the final layer(Number of classes)

    Returns:
      model: Returns the custom model added to ResNet
    """

    model = ResNet50(weights='imagenet', include_top=False,input_shape=(128,128,3))
    model.layers.pop()
    #model.layers.pop()
    #model.layers.pop()

    model.outputs = [model.layers[-1].output]

    #model.layers[-2].outbound_node= []
    x=Conv2D(256, kernel_size=(2,2),strides=2)(model.output)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)    
    x=Conv2D(128, kernel_size=(2,2),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)

    model=Model(model.input,x)

    for layer in model.layers[:15]:

        layer.trainable = False


    return model

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras import backend as K
num_classes=12
model_res = resnet_model(num_classes)
model_res.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model_res.summary()

#### **Training ResNet50**

In [ ]:
epochs = 10
batch_size = 32
model_checkpoint = ModelCheckpoint('resnet_weights.h5', monitor='val_accuracy', save_best_only=True,mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=0.000001)
#early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=7, verbose=0, mode='min', restore_best_weights=True)

model_res.fit(datagen.flow(X_train,Y_train),
          batch_size=128,
          epochs=20,
          verbose=1, shuffle=True, validation_data=(X_valid,Y_valid), callbacks=[model_checkpoint,reduce_lr])

#### **Visualizing Loss and Accuracy(ResNet50)**

In [ ]:
accuracy_curves(model_res)

In [ ]:
loss_curves(model_res)

#### **Classification Report**

In [ ]:
class_report(model_res)

#### **Confusion matrix**

In [ ]:
conf_matrix(model_res)

#### Model Optimization Using TF-Lite(Post Training Dynamic range quantization) 

In [ ]:
converter_resnet = tf.lite.TFLiteConverter.from_keras_model(model_res)


In [ ]:
# Convert to TF Lite without quantization
resnet_tflite_file = tflite_models_dir/"resnet.tflite"
resnet_tflite_file.write_bytes(converter_resnet.convert())

In [ ]:
interpreter = tf.lite.Interpreter(model_path='./tflite_models/resnet.tflite')
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
op = evaluate_model(resnet_tflite_file, model_type="Float")
print("Accuracy of TFLite - Resnet50 : {}".format(op))

#### Optimization of Resnet Model with TensorRT

In [ ]:
model_res.save('res_saved_model')

In [ ]:
print('Converting to TF-TRT FP16...')
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
    precision_mode=trt.TrtPrecisionMode.FP16,
    max_workspace_size_bytes=8000000000)
converter = trt.TrtGraphConverterV2(
   input_saved_model_dir='res_saved_model', conversion_params=conversion_params)
converter.convert()
converter.save(output_saved_model_dir='res_saved_model_TFTRT_FP16')
print('Done Converting to TF-TRT FP16')

In [ ]:
benchmark_tftrt('res_saved_model_TFTRT_FP16')

### **3) InceptionV3**

#### **Customizing InceptionV3 for our problem statement**

In [ ]:
def incep_model(num_classes=None):
    
    """ Adding custom model to the InceptionV3

    Args:
      num_classes: Number of layers in the final layer(Number of classes)

    Returns:
      model: Returns the custom model added to Inception
    """

    model = InceptionV3(weights='imagenet', include_top=False,input_shape=(128,128,3))
    #model.layers.pop()
    #model.layers.pop()
    #model.layers.pop()

#    for layer in model.layers:
 #     layer.trainable = False
    '''
    model.outputs = model.output
    
    x = GlobalAveragePooling2D()(model.outputs)
    #x=Conv2D(256, kernel_size=(2,2),strides=2)(model.output)
    #x = BatchNormalization()(x)    
    #x=Conv2D(128, kernel_size=(2,2),strides=1)(x)
    #x = Activation('relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(128,activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    #x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)

    model=Model(model.input,x)'''

    
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)

    predictions = Dense(12, activation='softmax')(x)

    model = Model(model.input, predictions)




    return model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K
num_classes=12
model_incep = incep_model(num_classes)
model_incep.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model_incep.summary()

#### **Training InceptionV3**

In [ ]:
epochs = 10
batch_size = 32
model_checkpoint = ModelCheckpoint('incep_weights.h5', monitor='val_accuracy', save_best_only=True,mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=0.000001)
#early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=7, verbose=0, mode='min', restore_best_weights=True)

model_incep.fit(datagen.flow(X_train,Y_train),
          batch_size=128,
          epochs=20,
          verbose=1, shuffle=True, validation_data=(X_valid,Y_valid), callbacks=[model_checkpoint])

#### **Visualizing Loss and Accuracy(InceptionV3)**

In [ ]:
accuracy_curves(model_incep)

In [ ]:
loss_curves(model_incep)

#### **Classification Report**

In [ ]:
class_report(model_incep)

#### **Confusion Matrix**

In [ ]:
conf_matrix(model_incep)

#### Model Optimization Using TF-Lite(Post Training Dynamic range quantization) 

In [ ]:
converter_incep = tf.lite.TFLiteConverter.from_keras_model(model_incep)

In [ ]:
# Convert to TF Lite without quantization
incep_tflite_file = tflite_models_dir/"incep.tflite"
incep_tflite_file.write_bytes(converter_incep.convert())

In [ ]:
interpreter = tf.lite.Interpreter(model_path='./tflite_models/incep.tflite')
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
op = evaluate_model(incep_tflite_file, model_type="Float")
print("Accuracy of TFLite - InceptionV3 : {}".format(op))

#### Optimization of InceptionV3 Model with TensorRT

In [ ]:
model_incep.save('incep_saved_model')

In [ ]:
benchmark_tftrt('incep_saved_model_TFTRT_FP16')

## **Saving,Loading and Predicting on test set**

### The validation accuracies obtained for each model are as follows:

**1. VGG-16 - 91.789**

**2. ResNet50 - 95.368**

**3. InceptionV3 - 89.895** 

As per the question, we choose the model with the highest validation accuracy for predictions on test set i.e ResNet50.

In [ ]:
!mkdir -p saved_models


In [ ]:
model.save('m_vgg.h5')
model_res.save('m_res.h5')
model_incep.save('m_incep.h5')

## **Saved Models**

[![Screenshot-2020-08-11-at-11-33-23-AM.png](https://i.postimg.cc/Ls9kkXD0/Screenshot-2020-08-11-at-11-33-23-AM.png)](https://postimg.cc/0z4JP9Lp)

In [ ]:
new_model = tf.keras.models.load_model('m_res.h5')

In [ ]:
new_model.summary()

In [ ]:
def predict_on_test(model):
    """ Using the Model with the highest Validation accuracy to predict on test set

    Args:
      model: Model with highest validation accuracy(ResNet50)

    Returns:
      df: A Dataframe in a format required by Kaggle 
    """
    prob=[]
    num=[]
    test_img=[]
    test_path = os.path.join('/kaggle/input/plant-seedlings-classification','test')
    test_all = fnmatch.filter(os.listdir(test_path), '*.png')

    test_img=[]
    for i in range(len(test_all)):
        path=test_path+'/'+test_all[i]
        temp_img=image.load_img(path,target_size=(128,128))
        temp_img=image.img_to_array(temp_img)
        test_img.append(temp_img) 
    test_img=np.array(test_img)    
    test_img= tf.keras.applications.vgg16.preprocess_input(test_img)


    test_labels=[]
    pred=model.predict(test_img)
    num2label =  {0:'Loose Silky-bent', 1:'Charlock',2: 'Sugar beet',3: 'Small-flowered Cranesbill',
                  4:'Common Chickweed',5: 'Common wheat',6: 'Maize', 7:'Cleavers', 8:'Scentless Mayweed',
                 9: 'Fat Hen', 10:'Black-grass', 11:'Shepherds Purse'}
    for i in range(len(test_all)):
        max_score =0
        lab=-1
        for j in range(12):
            if pred[i][j]>max_score:
                max_score=pred[i][j]
                lab=j
        test_labels.append(num2label[lab])


    d = {'file': test_all, 'species': test_labels}
    df = pd.DataFrame(data=d)
    return df

In [ ]:
df = predict_on_test(new_model)
print(df.head(5))
df.to_csv("/kaggle/working/submit.csv",index=False) 

# Score on test Set

Received a score of 0.94 on the ResNet50 model


[![Res-Net-Score.png](https://i.postimg.cc/j5Tb3Hph/Res-Net-Score.png)](https://postimg.cc/jWMmL7dW)